In [1]:
# settings:

cancer_type = '' # '' for multiclass
classifier_type = 'XGBoost'


In [4]:
# adding this path so we can import get_train_and_test
import sys
path = '../'
sys.path.append(path)
from get_train_and_test import get_train_and_test
root_path = path

seed = 42 # using a seed for splitting up the train and test data 

m_values_train, m_values_test, diagnoses_train, diagnoses_test = get_train_and_test(cancer_type, use_small=False, root_path = root_path, model_path = root_path + '/xgboost/', model_type = classifier_type + '_' + cancer_type, seed = seed)


# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
bst = xgb.XGBClassifier()


Getting values and diagnoses from: 
/Tank/methylation-patterns-code/methylation-patterns-izzy/data_preprocessing/dataset/pandas/m_values/TCGA-BLCA.csv
/Tank/methylation-patterns-code/methylation-patterns-izzy/data_preprocessing/dataset/pandas/diagnoses/TCGA-BLCA.csv
m_value and diagnoses shapes:
(278118, 436)
(436,)
m values train, m values test, diagnoses train, diagnoses test shapes:
(327, 278118) (109, 278118) (327,) (109,)


In [9]:

from scipy.stats import randint, uniform

# binary version:
# to_try = {'max_depth': randint(3, 13),
#           'n_estimators': randint(50, 500),
#           'learning_rate':uniform(0.001,0.5)}

to_try = {'max_depth': [3, 2],
         'n_estimators': [500, 800, 1200],
         'subsample': [0.5, 0.25],
          'colsample_bytree': [0.5, 0.25, 0.125],
          'objective':'binary:logistic',
          'learning_rate':0.189
         }

from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(bst, to_try, n_iter=5, scoring='f1_micro', cv=3)
random_search.fit(m_values_train, diagnoses_train)

random_search_results = random_search.cv_results_
random_search_best_estimator = random_search.best_estimator_

# showing parameters and scores
scores = random_search.cv_results_['mean_test_score']
params = random_search.cv_results_['params']
import pandas as pd
pd_results = pd.DataFrame(params)
pd_results["score"] = scores
pd_results.sort_values(by="score", ascending = False)

print(pd_results)

pd_results.to_csv('pd_results.csv')

# saving best model
params = random_search.best_params_
import json
params = json.dumps(params) # get string rep of dictionary
import joblib # joblib is apparently more efficient than pickle functions for model saving (see https://scikit-learn.org/stable/modules/model_persistence.html)
joblib.dump(random_search_best_estimator, 'saved_models/xgboost_model_multiclass_best_params_'+params+'.pkl')
booster = random_search_best_estimator.get_booster()
booster.dump_model('saved_models/xgboost_trees_multiclass_best_params_'+params+'.txt')


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
          

,learning_rate,max_depth,n_estimators,score
0,0.189,10,458,0.975535
1,0.189,10,436,0.975535
2,0.189,10,307,0.975535
3,0.189,10,248,0.975535
4,0.189,10,218,0.975535


   learning_rate  max_depth  n_estimators     score
0          0.189         10           458  0.975535
1          0.189         10           436  0.975535
2          0.189         10           307  0.975535
3          0.189         10           248  0.975535
4          0.189         10           218  0.975535


['saved_models/xgboost_model_multiclass_best_params_{"learning_rate": 0.189, "max_depth": 10, "n_estimators": 458}.pkl']